## Importing essential libraries

In [1]:
import tweepy
import pandas as pd
import numpy as np
from textblob import TextBlob
import re
from datetime import datetime, timedelta

## Keys and tokens for accessing tweets

In [2]:
ACCESS_TOKEN="802567534264389632-YaYdglNfenTBGmxTITX3SBDEPFlYzmT"
ACCESS_TOKEN_SECRET="ajNgI7XALZwOjkwX3i1bud8QoZNVXPGbJ7lbMS1c15P0E"
CONSUMER_KEY="iVDMrvc1G7UEczFtxN46Vo3n6"
CONSUMER_SECRET="AEtF5hGcUQWKB68Q45WK3pNXJJ6kJ40U8IhMMCRr4krFj65IU4"

## Twittter Authentication

In [3]:
try:
    def authentication(CONSUMER_KEY,CONSUMER_SECRET,ACCESS_TOKEN,ACCESS_TOKEN_SECRET):
        auth = tweepy.OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET)
        auth.set_access_token(ACCESS_TOKEN,ACCESS_TOKEN_SECRET)
        api = tweepy.API(auth)
        return api
except:
    print("Authentication Failed")

## Funtion to search tweets containing keywords

In [4]:
def search_tweets(keyword, total_tweets):
    today_datetime = datetime.today().now()
    yesterday_datetime = today_datetime - timedelta(days=1)
    today_date = today_datetime.strftime('%Y-%m-%d')
    yesterday_date = yesterday_datetime.strftime('%Y-%m-%d')
    api = authentication(CONSUMER_KEY,CONSUMER_SECRET,ACCESS_TOKEN,ACCESS_TOKEN_SECRET)
    
    tweets = tweepy.Cursor(api.search, q=keyword,since=yesterday_date,result_type='recent', lang='en').items(total_tweets)
    return tweets

## getting user input

In [5]:
keyword = str(input('Enter a keyword: '))

Enter a keyword: Narendra Modi


In [6]:
total_tweets = int(input("Enter total number of tweets: "))

Enter total number of tweets: 200


## Function call with keyword and total_tweets as parameters

In [7]:
tweets=search_tweets(keyword,total_tweets)

## Creating a dataframe of the aquired tweets

In [8]:
df=pd.DataFrame(data=[tweet.text for tweet in tweets],columns=['tweets'])

In [9]:
df.head()

,tweets
0,RT @srinivasiyc: Followed by PM Narendra Modi ...
1,"@ashoswai @HappymonJacob Ashok sir, once upon ..."
2,RT @Chandankrishn13: @HuXijin_GT Xi Jinping is...
3,RT @narendramodi_in: We are facing many a cris...
4,"RT @JIX5A: Sonia’s foot soldier Rajdeep Asks, ..."


## Cleaning tweets using regex

In [10]:
def clean_tweet(tweet):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

## Analysing tweet sentiment

In [11]:
def analyze_sentiment(tweet):
    analysis = TextBlob(clean_tweet(tweet))
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity == 0:
        return 'neutral'
    else:
        return 'negative'

## Adding clean tweets and sentiment to dataframe as columns

In [12]:
df['clean_tweets'] = df['tweets'].apply(lambda tweet:clean_tweet(tweet))

In [13]:
df['sentiment'] = df['tweets'].apply(lambda tweet:analyze_sentiment(tweet))

In [14]:
df.head()

,tweets,clean_tweets,sentiment
0,RT @srinivasiyc: Followed by PM Narendra Modi ...,RT Followed by PM Narendra Modi,neutral
1,"@ashoswai @HappymonJacob Ashok sir, once upon ...",Ashok sir once upon a time Former ISI Chief Gn...,negative
2,RT @Chandankrishn13: @HuXijin_GT Xi Jinping is...,RT GT Xi Jinping is the unelected ruler of an ...,neutral
3,RT @narendramodi_in: We are facing many a cris...,RT in We are facing many a crisis Is all this ...,positive
4,"RT @JIX5A: Sonia’s foot soldier Rajdeep Asks, ...",RT Sonia s foot soldier Rajdeep Asks why doesn...,neutral


## Calculating percentage of positive, negative and neutral tweets

In [15]:
total_positive_tweets = df[df['sentiment']=='positive']['tweets'].count()
total_negative_tweets = df[df['sentiment']=='negative']['tweets'].count()
total_neutral_tweets = df[df['sentiment']=='neutral']['tweets'].count()
percentage_positve_tweets = (100*total_positive_tweets)/total_tweets
percentage_negative_tweets = (100*total_negative_tweets)/total_tweets
percentage_neutral_tweets = (100*total_neutral_tweets)/total_tweets
print('Percentage of positive tweets are {}%'.format(percentage_positve_tweets))
print('Percentage of negative tweets are {}%'.format(percentage_negative_tweets))
print('Percentage of neutral tweets are {}%'.format(percentage_neutral_tweets))

Percentage of positive tweets are 30.0%
Percentage of negative tweets are 14.0%
Percentage of neutral tweets are 56.0%
